In [7]:
import pandas as pd

title_akas = pd.read_csv('../downloads/title.akas.tsv', delimiter='\t')
title_basics = pd.read_csv('../downloads/title.basics.tsv', delimiter='\t')
title_crew = pd.read_csv('../downloads/title.crew.tsv', delimiter='\t')
title_ratings = pd.read_csv('../downloads/title.ratings.tsv', delimiter='\t')

In [5]:
#title_akas.drop(labels=['ordering', 'types', 'attributes'], axis=1)
#title_basics.drop(labels=['ordering', 'types', 'attributes'], axis=1)
#title_crew.drop(labels=['ordering', 'types', 'attributes'], axis=1)
#title_ratings.drop(labels=['ordering', 'types', 'attributes'], axis=1)

,titleId,title,region,language,isOriginalTitle
0,tt0000001,Карменсіта,UA,\N,0
1,tt0000001,Carmencita,DE,\N,0
2,tt0000001,Carmencita - spanyol tánc,HU,\N,0
3,tt0000001,Καρμενσίτα,GR,\N,0
4,tt0000001,Карменсита,RU,\N,0
...,...,...,...,...,...
33341644,tt9916852,Episódio #3.20,PT,pt,0
33341645,tt9916852,Episodio #3.20,IT,it,0
33341646,tt9916852,एपिसोड #3.20,IN,hi,0
33341647,tt9916856,The Wind,DE,\N,0


In [6]:
title_basics.columns

NameError: name 'title_basics' is not defined

# TODO:
- remover filmes mais antigos que x data
- filtrar filmes apenas em en
- remover filmes sem sinopse (isto é quando formos fazer o webscrapping ig)
